In [1]:
import pandas as pd

**Merging CSV from Coll to Merged CSV from ArrayAccess and BPS-based-original data**

In [2]:
base_regency = pd.read_csv('output/base-aa/merge-regency.csv')
base_village = pd.read_csv('output/base-aa/merge-village.csv')

geocode_regency = pd.read_csv('data/coll/kecamatan_lat_long.csv')
geocode_village = pd.read_csv('data/coll/kelurahan_lat_long.csv')

In [3]:
OUTPATH_REGENCY = 'output/base-coll/merge-regency.csv'
OUTPATH_VILLAGE = 'output/base-coll/merge-village.csv'

def regency_with_geocode():
  merged_df = pd.merge(base_regency, geocode_regency, how='left', left_on='id', right_on='ids')

  merged_df['Latitude'] = merged_df['Latitude'].fillna(merged_df['lat'])
  merged_df['Longitude'] = merged_df['Longitude'].fillna(merged_df['long'])
  merged_df.drop(['id_y', 'ids', 'foreign', 'name_y', 'lat', 'long'], axis=1, inplace=True)
  merged_df = merged_df.rename(columns = {"id_x": "id", "name_x": "name"})
  merged_df.to_csv(OUTPATH_REGENCY, index=False)

def village_with_geocode():
  merged_df = pd.merge(base_village, geocode_village, how='left', left_on='id', right_on='id')

  merged_df['Latitude'] = merged_df['Latitude'].fillna(merged_df['lat'])
  merged_df['Longitude'] = merged_df['Longitude'].fillna(merged_df['long'])
  merged_df.drop(['name_y', 'Unnamed: 0', 'foreign', 'name_y', 'lat', 'long'], axis=1, inplace=True)
  merged_df = merged_df.rename(columns = {"name_x": "name"})
  merged_df.to_csv(OUTPATH_VILLAGE, index=False)



In [4]:
# regency_with_geocode()
# village_with_geocode()

**Checking Data on Merged CSV**

In [5]:
df_merge_regency_v1 = pd.read_csv(OUTPATH_REGENCY)
# df_merge_regency_v1.head()
df_merge_regency_v1.isnull().sum()

id            0
parent_id     0
name          0
Latitude     62
Longitude    62
dtype: int64

In [6]:
df_merge_village_v1 = pd.read_csv(OUTPATH_VILLAGE)
# df_province.head()
df_merge_village_v1.isnull().sum()

id                0
parent_id         0
kode_dagri        0
name              0
Latitude      24503
Longitude     24503
dtype: int64

**Filling the missing value**

For the regency, collect the lat and long value of the same district and count the mean. The mean value will be used to fill missing value.
For the village, collect the lat and long value of the same regency and count the mean. The mean value will be used to fill missing value.

In [7]:
from pandas import DataFrame

OUTPATH_REGENCY_v2 = 'output/base-coll/merge-regency-v2.csv'
OUTPATH_VILLAGE_v2 = 'output/base-coll/merge-village-v2.csv'

def fillMissingValue(df: DataFrame, outpath: str):
    mean_values = df.groupby('parent_id')[['Latitude', 'Longitude']].mean()
    # Mendapatkan daftar parent_id yang memiliki nilai null
    null_parent_ids = df[df['Latitude'].isnull()]['parent_id'].unique()

    # Mengisi nilai null pada 'Latitude' dan 'Longitude' dengan nilai rata-rata yang dihitung, hanya untuk baris dengan parent_id yang sesuai
    for parent_id in null_parent_ids:
        mean_lat = mean_values.loc[parent_id, 'Latitude']
        mean_long = mean_values.loc[parent_id, 'Longitude']
        mask = (df['parent_id'] == parent_id) & df['Latitude'].isnull()
        df.loc[mask, 'Latitude'] = mean_lat
        df.loc[mask, 'Longitude'] = mean_long
    # Simpan hasil ke file CSV
    df.to_csv(outpath, index=False)

In [8]:
# fillMissingValue(df_merge_regency_v1, OUTPATH_REGENCY_v2)
# fillMissingValue(df_merge_village_v1, OUTPATH_VILLAGE_v2)

In [9]:
df_merge_regency_v2 = pd.read_csv(OUTPATH_REGENCY_v2)
# df_merge_regency_v2.head()
df_merge_regency_v2.isnull().sum()

id           0
parent_id    0
name         0
Latitude     0
Longitude    0
dtype: int64

In [10]:
df_merge_village_v2 = pd.read_csv(OUTPATH_VILLAGE_v2)
# df_merge_village_v2.head()
df_merge_village_v2.isnull().sum()

id               0
parent_id        0
kode_dagri       0
name             0
Latitude      6210
Longitude     6210
dtype: int64

village still have missing value. Take the lat long value from the regency and give it to the villages

In [11]:
PATH_BASE_VALUE = OUTPATH_REGENCY_v2
OUTPATH_VILLAGE_v3 = 'output/base-coll/merge-village-v3.csv'
df1 = df_merge_village_v2
df2 = pd.read_csv(PATH_BASE_VALUE)

parent_lat_long_map = df2.set_index('id')[['Latitude', 'Longitude']].to_dict(orient='index')

for index, row in df1.iterrows():
    if pd.isnull(row['Latitude']) or pd.isnull(row['Longitude']):
        parent_id = row['parent_id']
        if parent_id in parent_lat_long_map:
            lat_long = parent_lat_long_map[parent_id]
            df1.at[index, 'Latitude'] = lat_long['Latitude']
            df1.at[index, 'Longitude'] = lat_long['Longitude']

df1.to_csv(OUTPATH_VILLAGE_v3, index=False)

In [12]:
df_merge_village_v3 = pd.read_csv(OUTPATH_VILLAGE_v3)
# df_merge_village_v3.head()
df_merge_village_v3.isnull().sum()

id            0
parent_id     0
kode_dagri    0
name          0
Latitude      0
Longitude     0
dtype: int64